In [2]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

def load_data(dataset_dir, max_sequence_length=30):
    keypoints = []
    labels = []
    label_map = {}
    label_counter = 0
    
    for category in os.listdir(dataset_dir):
        category_path = os.path.join(dataset_dir, category)
        if os.path.isdir(category_path):
            if category not in label_map:
                label_map[category] = label_counter
                label_counter += 1
            label = label_map[category]

            for file in os.listdir(category_path):
                file_path = os.path.join(category_path, file)
                if file.endswith('.npy'):
                    keypoints_data = np.load(file_path)
                    if keypoints_data.size > 0:
                        keypoints.append(keypoints_data)
                        labels.append(label)

    keypoints = pad_sequences(keypoints, maxlen=max_sequence_length, padding='post', truncating='post')
    keypoints = keypoints.reshape(keypoints.shape[0], keypoints.shape[1], -1)
    
    return np.array(keypoints), np.array(labels), label_map

def preprocess_data(keypoints, labels):
    keypoints = keypoints / np.max(keypoints)  # Normalize keypoints
    labels = to_categorical(labels)  # Convert labels to categorical
    return keypoints, labels

def build_lstm_model(input_shape, num_classes):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=input_shape))  
    model.add(LSTM(64))  
    model.add(Dense(num_classes, activation='softmax'))  
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_lstm_model(dataset_dir, model_save_path='sign_language_model.h5'):
    keypoints, labels, label_map = load_data(dataset_dir)
    keypoints, labels = preprocess_data(keypoints, labels)

    X_train, X_test, y_train, y_test = train_test_split(keypoints, labels, test_size=0.2, random_state=42)

    input_shape = (X_train.shape[1], X_train.shape[2])  
    num_classes = len(label_map)
    model = build_lstm_model(input_shape, num_classes)
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))
    
    model.save(model_save_path)
    print(f"Model saved at {model_save_path}")

# Run the training process
dataset_dir = r'Keypoints/Images'  
train_lstm_model(dataset_dir)


Epoch 1/100
25/25 [==============================] - 12s 128ms/step - loss: 4.6112 - accuracy: 0.0866 - val_loss: 4.4138 - val_accuracy: 0.1218
Epoch 2/100
25/25 [==============================] - 1s 54ms/step - loss: 4.3959 - accuracy: 0.0917 - val_loss: 4.3342 - val_accuracy: 0.1218
Epoch 3/100
25/25 [==============================] - 1s 53ms/step - loss: 4.3476 - accuracy: 0.0917 - val_loss: 4.3269 - val_accuracy: 0.0914
Epoch 4/100
25/25 [==============================] - 1s 58ms/step - loss: 4.3376 - accuracy: 0.1032 - val_loss: 4.3214 - val_accuracy: 0.0914
Epoch 5/100
25/25 [==============================] - 1s 53ms/step - loss: 4.3287 - accuracy: 0.0841 - val_loss: 4.3141 - val_accuracy: 0.1218
Epoch 6/100
25/25 [==============================] - 1s 53ms/step - loss: 4.3289 - accuracy: 0.0917 - val_loss: 4.3149 - val_accuracy: 0.1218
Epoch 7/100
25/25 [==============================] - 1s 58ms/step - loss: 4.3314 - accuracy: 0.1019 - val_loss: 4.3194 - val_accuracy: 0.1218
Epoc